In [6]:
import torch
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image

# Load the fine-tuned YOLOv5 model (from the location of the trained weights)
yolo_model = torch.hub.load('ultralytics/yolov5', 'custom', path='C:/yolov5/runs/train/exp6/weights/best.pt')

# Load the pre-trained MobileNetV2 model (for full/empty classification)
classification_model = load_model(r"c:/bin/trash_bin_model_epoch10.keras")

# Function to detect trash bins using the fine-tuned YOLOv5
def detect_bins(image):
    # Detect objects in the image using YOLOv5
    results = yolo_model(image)
    boxes = results.xyxy[0]  # Extract bounding boxes# Detect objects in the image using YOLOv5
    print(results.pandas().xyxy[0])  # Print detection results for debugging
    return boxes

# Function to classify bins (full/empty) using MobileNetV2
def classify_bin(cropped_image):
    resized_img = cv2.resize(cropped_image, (224, 224))
    img_array = np.expand_dims(resized_img, axis=0) / 255.0  # Normalize
    prediction = classification_model.predict(img_array)
    return prediction[0] > 0.5  # Return True for full, False for empty

# Function to process an uploaded image
def process_image(image_path):
    # Read the input image
    img = cv2.imread(image_path)

    # Detect trash bins using the fine-tuned YOLOv5
    boxes = detect_bins(img)

    # Create a copy of the image to draw bounding boxes on
    output_img = img.copy()

    for box in boxes:
        x1, y1, x2, y2 = map(int, box[:4])  # Get box coordinates and cast to int

        # Crop the detected bin from the image
        cropped_bin = img[y1:y2, x1:x2]

        # Classify the cropped bin as full or empty
        is_full = classify_bin(cropped_bin)

        # Draw rectangle and label
        color = (0, 255, 0) if is_full else (0, 0, 255)  # Green for full, Red for empty
        cv2.rectangle(output_img, (x1, y1), (x2, y2), color, 2)

        # Add label (General Waste/Plastic) and Full/Empty
        label_text = f"Full" if is_full else f"Empty"
        cv2.putText(output_img, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Save the processed image to a file
    processed_image_path = r"c:/bin/processed.png"
    cv2.imwrite(processed_image_path, output_img)
    print(f"Processed image saved as {processed_image_path}")

# Test the process with a sample image
process_image(r"c:/bin/new_image.png")


Using cache found in C:\Users\user/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-18 Python-3.11.3 torch-2.5.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
C:\Users\user/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


        xmin      ymin        xmax        ymax  confidence  class  \
0  14.563878  5.281422  478.322784  565.548035    0.752731      0   

                     name  
0  general wastetrash bin  
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 649ms/step
Processed image saved as c:/bin/processed.png
